In [5]:
!pip install langchain faiss-cpu transformers sentence-transformers accelerate pandas ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from typing import Any, List, Mapping, Optional

import langchain
import langchain.document_loaders
import pandas as pd
import torch
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain, chains, prompts
from langchain.document_loaders import DataFrameLoader
from langchain.document_transformers import LongContextReorder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models.llms import LLM
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline

## Полезные ресурсы

1. https://habr.com/ru/articles/779526/ - что вообще делаем
2. https://habr.com/ru/articles/599673/ - что такое GPT, как работает. Вам особенно пригодится пункт с файнтьюнингом
3. https://python.langchain.com/docs/get_started/introduction - основной фреймворк для создания RAG
4. https://www.llamaindex.ai/ - тоже фреймворк с похожим функционалом, может пригодиться, если langchain не покроет потребности все
5. https://huggingface.co/ - сайт с моделями

### Загрузка документов

В данной ячейке загружается база знаний. Сейчас тут текст, скопированный из википедии по астра линуксу. Тут могут быть документы в формате docx, pptx, pdf, pandas dataframe и так далее.

In [10]:
text = TextLoader("./content/astra_wiki.txt")

### Разбиваем документы на части

In [11]:
documents = text.load()
documents[0]

Document(page_content='Система работает с пакетами на базе .deb, используется пакетный менеджер apt. Astra Linux считается официально признанным деривативом Debian после прохождения необходимых проверок на соответствие требованиям Debian[9], АО «НПО РусБИТех» находится в партнёрских отношениях с The Linux Foundation[10] и The Document Foundation[11]. Разработчик заявляет, что «лицензионные соглашения на операционные системы Astra Linux разработаны в строгом соответствии с положениями действующих правовых документов Российской Федерации, а также международных правовых актов», при этом они «не противоречат духу и требованиям лицензии GPL».\n\nСобственный репозиторий Astra Linux состоит из более чем 20 000 пакетов. На пакетной базе этого репозитория развиваются и другие программные продукты компании[12]. В состав дистрибутива входят такие пакеты с открытым исходным кодом, как графическое окружение пользователя Fly, офисный пакет LibreOffice, доработанный в части управления доступом, серве

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=200)  # у langchain есть много способов разбить документы на чанки, можете изучить другие и подобрать оптимальный
texts = text_splitter.split_documents(documents)

### Вычисляем эмбеддинги для всех фрагментов

In [13]:
EMBEDDER_PATH = "intfloat/multilingual-e5-small"  # путь до модели эмбеддера, на huggingface можете подобрать другой, есть различные статьи по сравнению эмбеддеров

In [14]:
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDER_PATH
)

sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

384

### Cохраняем эмбеддинги в векторную БД

In [15]:
# создаем хранилище
db = FAISS.from_documents(texts, embeddings)  # faiss - одна из наиболее популярных векторных БД, можете заменить на любую другую доступную
db.as_retriever()

# также можно сохранить хранилище локально
db.save_local('faiss_index')

In [16]:
# тестируем ретривер
q = "Special Edition"  #  запрос, который ищем в векторной базе данных
db.similarity_search_with_score(q)  #  результат поиска и значение близости между запросом и документами в БД

[(Document(page_content='Common Edition — единственная российская ОС, репозиторий которой размещен в открытом доступе международной некоммерческой организации The Linux Foundation;\nSpecial Edition — сертифицированная ОС со встроенными средствами защиты информации (СЗИ) для стабильных и безопасных ИТ-инфраструктур любого масштаба и бесперебойной работы с данными любой степени конфиденциальности, лицензируется по трем уровням защиты.', metadata={'source': './content/astra_wiki.txt'}),
  0.30254507),
 (Document(page_content='Применение', metadata={'source': './content/astra_wiki.txt'}),
  0.32562888),
 (Document(page_content='Особенности версии Special Edition\nРежимы защищенности\nС релиза 2021 года (1.7/4.7) в Astra Linux Special Edition доступны режимы защищенности «Базовый» («Орел», несертифицированная версия), «Усиленный» («Воронеж») и «Максимальный» («Смоленск»). Режим «Усиленный» имеет все возможности режима «Базовый» и дополняет их, режим «Максимальный» имеет все возможности режи

In [17]:
retriever = db.as_retriever(search_kwargs={"k": 5})  # поиск К наиболее похожих документов на запрос
res = retriever.get_relevant_documents(q)
for x in res:
    print(x.page_content)
    print("*" * 50)

Common Edition — единственная российская ОС, репозиторий которой размещен в открытом доступе международной некоммерческой организации The Linux Foundation;
Special Edition — сертифицированная ОС со встроенными средствами защиты информации (СЗИ) для стабильных и безопасных ИТ-инфраструктур любого масштаба и бесперебойной работы с данными любой степени конфиденциальности, лицензируется по трем уровням защиты.
**************************************************
Применение
**************************************************
Особенности версии Special Edition
Режимы защищенности
С релиза 2021 года (1.7/4.7) в Astra Linux Special Edition доступны режимы защищенности «Базовый» («Орел», несертифицированная версия), «Усиленный» («Воронеж») и «Максимальный» («Смоленск»). Режим «Усиленный» имеет все возможности режима «Базовый» и дополняет их, режим «Максимальный» имеет все возможности режима «Усиленный» и также дополняет их.
**************************************************
Основные версии
 Внешн

### Подключаем Большую Языковую Модель

Посмотрите какие еще есть GPT модели в открытом доступе с небольшим кол-вом параметров (до 1 млрда, свыше 500 млн вряд ли поместится в collab)

Помимо GPT моделей рекомендую рассмотреть архитектуру T5. В рамках кейса Вас будет интересовать способность модели отвечать на вопросы (QA answering). Пример: https://habr.com/ru/articles/581932/

In [18]:
MODEL_NAME = r'ai-forever/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)  # загружаем токенизатор модели

In [19]:
class CustomLLM(LLM):
    model: object
    tokenizer: object

    @property
    def _llm_type(self) -> str:
        return "custom"

    def inference_llama(self, prompt: str):
        # Encode the prompt and generate tokens
        inputs = self.tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(device)
        generated_ids = self.model.generate(**inputs, max_new_tokens=1024, num_beams=1, do_sample=False)
        outputs = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        return outputs[0]

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:

        return self.inference_llama(prompt)

In [20]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='auto')  #  загружаем выбранную модель
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # при исползовании GPU скорость работы значительно возрастает. Не рекомендую оставлять сессию с включенным GPU  в бездействии.
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [21]:
llm = CustomLLM(model=model, tokenizer=tokenizer)

In [22]:
llm("Как звали Достоевского?")

/home/sinredempti0n/project/.venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Как звали Достоевского?\nНиколай Васильевич Гоголь\n\n\n\n\nНиколай Васильевич Гоголь\n\nКак звали Достоевского?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

### Собираем Retrieval-Augmented Generation

В ячейке ниже предоставлен промпт для работы с большими языковыми моделями. Мы работали с моделью с 7 млрдами параметров. В Вашем кейсе, может быть, промпт нужно будет немного видоизменить в зависимости от модели, которую вы выберете.

In [23]:
# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Текст:
-----
{context}
-----

Пожалуйста, посмотри на текст выше и ответь на вопрос, используя информацию из этого текста.

Вопрос:
{query}

"""

prompt = prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = chains.LLMChain(llm=llm, prompt=prompt)
chain = chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

In [24]:
QUESTION = "Что такое Special Edition?"  # Вопрос, на который мы хотим получить ответ
NUMBER_OF_DOCS_TO_RETRIEVE = 5  # кол-во документов, которые мы хотим достать из векторной БД для ответа на вопрос

In [25]:
print(llm(QUESTION))  # ответ модели без использования RAG (без контекста из базы знаний)

Что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты читаешь книгу, а потом смотришь фильм.

— А что такое Special Edition?

— Это когда ты 

In [26]:
retriever = db.as_retriever(search_kwargs={"k": NUMBER_OF_DOCS_TO_RETRIEVE})

In [27]:
reorderer = LongContextReorder()


def answer(query, reorder=True, print_results=False):
    results = retriever.get_relevant_documents(query)
    if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
    if reorder:
        results = reorderer.transform_documents(results)

    generated_text = chain.run(input_documents=results, query=query).split("Ответ:")[-1]
    return generated_text

In [28]:
gen_text = answer(QUESTION, print_results=False)  # Ответ модели с учетом используемого контекста. Обратите внимание на неспособность модели хорошо отвечать на вопрос. В этом поможет дообучение модели на предоставленных ранее данных (пары вопрос-ответ).
print(str(gen_text))

/home/sinredempti0n/project/.venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Это специальная версия, которая предназначена для использования в системах защиты информации, в том числе в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступом, в системах управления доступо